![title](./image/title.png)

## Scenario: Flexible Notification Service

**Problem:** You're building a notification service for a food delivery application. Initially, the service only sends email notifications. You want to extend it to support multiple notification channels (WhatsApp, Facebook, SMS, etc.) without creating a combinatorial explosion of classes.

**Initial Design (Problematic):**

*   `Notifier` class sends email notifications.
*   To add WhatsApp and Facebook notifications, you create `WhatsappNotifier` and `FacebookNotifier` subclasses.
*   This becomes unwieldy when a customer wants to receive notifications via multiple channels (e.g., both WhatsApp *and* Facebook).  You'd have to create new classes for every combination (e.g., `WhatsappFacebookNotifier`).
*   Adding a new notification channel (e.g., SMS) drastically increases the number of required classes.

**The Challenge:**

*   How to add new notification channels easily without modifying the core `Notifier` class.
*   How to allow customers to receive notifications via multiple channels simultaneously without creating a massive hierarchy of classes.

**Proposed Solution:**

Use the **Decorator Pattern** to dynamically add notification channels to the core `Notifier` object.

1.  **Core Component:** Keep the `Notifier` class responsible for email notifications.
2.  **Interface:** Define an `INotifier` interface that the `Notifier` class implements.
3.  **Base Decorator:** Create a `BaseNotifierDecorator` class that also implements `INotifier`.  This decorator will hold a reference to another `INotifier` (either the core `Notifier` or another decorator).
4.  **Concrete Decorators:** Create `FacebookDecorator`, `WhatsappDecorator`, etc., which extend `BaseNotifierDecorator`.  Each decorator adds a specific notification channel. They will implement the `send` method in a way that calls the underlying component's `send` method *and* then adds the specific notification logic.

This allows you to chain decorators to achieve multiple notification channels (e.g., `FacebookDecorator(WhatsappDecorator(Notifier()))` will send notifications via email, WhatsApp, and Facebook). This approach avoids the explosion of subclasses.

## Problematic Initial Design: Notification Service (Before Decorator Pattern)

The initial design uses inheritance to add notification channels, which leads to several problems.

In [ ]:
class DatabaseService:
    def get_mail_from_username(self, username):
        return username + "@Mail"

    def get_phone_nbr_from_username(self, username):
        return username + "@Phone"

    def get_fb_name_from_username(self, username):
        return username + "@Facebook"

In [ ]:
class Notifier:
    def __init__(self, username):
        self.username = username
        self.database_service = DatabaseService()

    def send(self, msg):
        mail = self.database_service.get_mail_from_username(self.username)
        print(f"Sending {msg} by Mail to {mail}")

In [ ]:
class FacebookNotifier(Notifier):
    def __init__(self, username):
        super().__init__(username)

    def send(self, msg):
        fb_name = self.database_service.get_fb_name_from_username(self.username)
        print(f"Sending {msg} on Facebook to {fb_name}")

In [ ]:
class WhatsappNotifier(Notifier):
    def __init__(self, username):
        super().__init__(username)

    def send(self, msg):
        phone_number = self.database_service.get_phone_nbr_from_username(self.username)
        print(f"Sending {msg} by WhatsApp on {phone_number}")

**Why This Approach is Flawed**

![inheritance tree](./image/inheritance_tree.png)

1.  **Combinatorial Explosion of Classes:**  The diagram with the inheritance tree clearly shows this problem.  If you want to support multiple notification channels *simultaneously* (e.g., WhatsApp and Facebook), you'll need to create a new class for *every possible combination*: `WhatsappAndFacebookNotifier`, `WhatsappAndSMSNotifier`, `FacebookAndSMSNotifier`, `WhatsappAndFacebookAndSMSNotifier`, and so on.  Adding even one new notification channel dramatically increases the number of classes required.  This becomes unmaintainable very quickly.

2.  **Limited Flexibility:** The client code can only choose *one* notification method at a time.  A customer can either be notified via WhatsApp *or* Facebook *or* email, but not a combination of them, unless you create a specific class for that combination.

3.  **Code Duplication:** Although not explicitly shown in this simplified code, there's likely to be some code duplication in the `send` methods of the various notifier classes. Each subclass has to handle the specific logic for its notification channel.

4.  **Violation of Open/Closed Principle:** To add a new notification channel, you have to modify the existing class hierarchy by creating a new subclass.  Ideally, you should be able to add new functionality without modifying existing code (Open/Closed Principle).

5.  **Rigidity:** It becomes difficult to change the notification strategy at runtime. You'd have to instantiate a different class, which might require significant code changes in the client.

**In summary:** The inheritance-based approach quickly becomes unmanageable, inflexible, and violates design principles when you need to support multiple, combinable notification channels. The Decorator pattern provides a much more elegant and flexible solution.

![Example](./image/example.png)

In [ ]:
from abc import ABC, abstractmethod

# 1. Component Interface
class INotifier(ABC):
    """
    Interface for notification services.
    """
    @abstractmethod
    def send(self, msg):
        pass

    @abstractmethod
    def get_username(self):
        pass

# 2. Concrete Component
class Notifier(INotifier):
    """
    The core notification service (sends email).
    """
    def __init__(self, username):
        self.username = username
        self.database_service = DatabaseService()

    def send(self, msg):
        mail = self.database_service.get_mail_from_username(self.username)
        print(f"Sending {msg} by Mail to {mail}")

    def get_username(self):
        return self.username

# 3. Base Decorator
class BaseNotifierDecorator(INotifier):
    """
    Base class for notification decorators.
    """
    def __init__(self, wrapped):
        self.wrapped = wrapped
        self.database_service = DatabaseService()

    @abstractmethod
    def send(self, msg):
        pass

    def get_username(self):
        return self.wrapped.get_username()

# 4. Concrete Decorators
class WhatsappDecorator(BaseNotifierDecorator):
    """
    Decorator for sending WhatsApp notifications.
    """
    def send(self, msg):
        self.wrapped.send(msg)
        phone_number = self.database_service.get_phone_nbr_from_username(self.get_username())
        print(f"Sending {msg} by WhatsApp on {phone_number}")


class FacebookDecorator(BaseNotifierDecorator):
    """
    Decorator for sending Facebook notifications.
    """
    def send(self, msg):
        self.wrapped.send(msg)
        fb_name = self.database_service.get_fb_name_from_username(self.get_username())
        print(f"Sending {msg} on Facebook to {fb_name}")

# 5. (Simplified) Database Service
class DatabaseService:
    """
    Simulates a database service for retrieving user information.
    """
    def get_mail_from_username(self, username):
        return username + "@Mail"

    def get_phone_nbr_from_username(self, username):
        return username + "@Phone"

    def get_fb_name_from_username(self, username):
        return username + "@Facebook"


# Example Usage
if __name__ == "__main__":
    # Create a core notifier
    notifier = Notifier("Geekific")

    # Decorate it with WhatsApp and Facebook
    decorated_notifier = FacebookDecorator(WhatsappDecorator(notifier))

    # Send a message
    decorated_notifier.send("Give this repo star!!!")